In [1]:
import pyodbc
import pandas as pd
server = 'nombre_servidor'  # Cambia por tu servidor
database = 'nombre_base_datos'  # Cambia por tu base de datos
username = 'usuario'  # Cambia por tu usuario
password = 'contraseña'  # Cambia por tu contraseña

# Crear conexión
connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER=localhost\SQLEXPRESS01;DATABASE=datamart_importaciones_2;Trusted_Connection=yes;"
conn = pyodbc.connect(connection_string)

In [2]:
query = "SELECT * FROM Dim_Mercaderia"

In [3]:
df = pd.read_sql(query,conn)

In [4]:
# Preprocesamiento
import re 
def preprocess_text(text):
    text = re.sub(r"[^a-zA-Z\s]", " ", text)  # Quitar caracteres especiales
    text = text.lower().strip()  # Minúsculas y eliminar espacios extra
    return text


In [5]:
df['limpio'] = df['mercaderia'].apply(lambda x:x.replace(", S/M","").replace(",S/M",""))
df['limpio'] = df['limpio'].apply(lambda x:preprocess_text(x))

In [6]:
from gensim.models import Word2Vec

# Cargar el modelo Word2Vec preentrenado
w2v_model = Word2Vec.load("w2v_model.model")
# Verifica la dimensión de los vectores
vector_size = w2v_model.vector_size
print(f"Dimensión del vector: {vector_size}")

# Función para calcular el promedio del embedding de una frase
def calculate_average_embedding(sentence, model):
    tokens = sentence.split()  # Tokenizar la oración
    word_vectors = [model.wv[word] for word in tokens if word in model.wv]  # Filtrar palabras fuera del vocabulario
    if not word_vectors:  # Si no hay palabras en el vocabulario, retornar un vector de ceros
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)  # Promediar los vectores de las palabras


Dimensión del vector: 100


In [7]:
import pandas as pd
import numpy as np
# Calcular los embeddings promedio para cada texto
df['embedding'] = df['limpio'].apply(lambda x: calculate_average_embedding(x, w2v_model))

In [8]:
vector_matrix = np.array(df['embedding'].tolist())

In [9]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=30, random_state=42,n_init=5)
df['etiqueta'] = kmeans.fit_predict(vector_matrix)

In [14]:
import pandas as pd
from collections import Counter

# Suponiendo que tu dataframe se llama `df` y tiene columnas 'oraciones' y 'etiquetas'

# Iterar sobre cada etiqueta única
for etiqueta in df['etiqueta'].unique():
    # Filtrar las oraciones asociadas a la etiqueta
    oraciones = df.loc[df['etiqueta'] == etiqueta, 'limpio']
    
    # Unir todas las palabras de las oraciones en una lista
    palabras = " ".join(oraciones).split()
    palabras = [i for i in palabras if len(i)>3]
    
    # Contar la frecuencia de cada palabra
    contador = Counter(palabras)
    
    # Obtener las 20 palabras más repetidas
    top_20_palabras = [palabra for palabra, _ in contador.most_common(20)]
    
    # Convertir las palabras a una cadena separada por comas
    palabras_como_string = ", ".join(top_20_palabras)
    
    # Imprimir el resultado para el cluster actual
    print(f"Cluster {etiqueta}: {palabras_como_string}")

Cluster 25: valvula, bomba, agua, manguera, tubo, para, combustible, motor, tanque, electrico, hidraulica, escape, ventilador, aire, tapa, tuberia, aceite, valvulas, hidraulico, electrica
Cluster 9: aceite, para, plato, taza, vaso, sabor, masglo, mango, crema, polvo, filtro, tinto, vino, marca, leche, miniso, bebida, bomba, base, botella
Cluster 18: para, reparacion, sellos, piston, mantenimiento, embrague, valvula, caterpillar, modelo, motor, empaque, bomba, cilindro, montaje, servicio, juntas, filtro, bajaj, sello, accesorios
Cluster 4: cable, para, mini, soporte, pantalla, conector, plus, marca, adaptador, lenovo, sensor, tarjeta, teclado, disco, bateria, computadora, celular, panel, switch, digital
Cluster 12: filtro, aire, para, interruptor, freno, valvula, agua, presion, aceite, sensor, elemento, modelo, combustible, bomba, pastilla, tubo, encendido, bobina, control, protector
Cluster 3: para, puerta, freno, posterior, brazo, honda, cubierta, tapa, amortiguador, inferior, manija,

In [15]:
# Diccionario con las etiquetas descriptivas por cluster
etiquetas_descriptivas = {
    25: "Sistema Hidráulico y Componentes",
    9: "Alimentos y Bebidas",
    18: "Reparación y Mantenimiento",
    4: "Electrónica y Accesorios",
    12: "Filtros y Sensores Automotrices",
    3: "Partes de Vehículos",
    8: "Juguetes y Decoración",
    21: "Ropa y Accesorios de Marca",
    13: "Libros y Herramientas",
    28: "Joyas y Accesorios",
    26: "Conexiones y Componentes Técnicos",
    19: "Textiles y Confecciones",
    22: "Herramientas y Placas",
    0: "Equipo Deportivo y Vehículos",
    24: "Soportes y Componentes Técnicos",
    2: "Tecnología y Componentes Eléctricos",
    15: "Materiales y Herramientas de Construcción",
    14: "Componentes Industriales",
    6: "Ropa y Accesorios Generales",
    10: "Bolsos y Mochilas",
    11: "Accesorios Electrónicos",
    1: "Productos Técnicos Diversos",
    16: "Partes Automotrices",
    7: "Componentes Caterpillar",
    27: "Moda y Accesorios Personales",
    17: "Motores y Automóviles",
    20: "Textiles y Ropa",
    5: "Calzado y Moda",
    29: "Ropa Interior y Pijamas",
    23: "Componentes Industriales Caterpillar"
}

# Agregar una nueva columna 'descripcion' al DataFrame con el mapeo
df['categoria'] = df['etiqueta'].map(etiquetas_descriptivas)

In [17]:
df_pepito = df[['id_Mercaderia','categoria']]

In [19]:
df_pepito.to_csv("categorias.csv",index=False)